In [ ]:
%pylab inline

In [ ]:
import numpy as np

import xarray as xr

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers

In [ ]:
inputs = xr.open_dataset("wd/ml/X.nc").isel(x=0)
rho = xr.open_dataset("wd/stat.nc").RHO[-1]

In [ ]:
from gnl.data_matrix import DataMatrix

In [ ]:
inputs

In [ ]:
dm = DataMatrix(['z'], ['time'], ['qt'])
X = dm.dataset_to_mat(inputs)

In [ ]:
X -= X.mean(axis=1)[:,None]
D = np.diag(np.sqrt(rho))
# D = np.eye(X.shape[1])

Weighted data

In [ ]:
Xw = X@ np.sqrt(D)

# Recreating SVD using neural networks

simple keras hidden layer autoencoder with MSE loss function. This should return the same output as EOFs.

In [ ]:
from keras.optimizers import Adam

In [ ]:
mod = Sequential()


nhidden = 4
mod.add(Dense(nhidden, input_dim=X.shape[1], kernel_regularizer=regularizers.l2(0.0)))
# mod.add(Activation('relu'))
mod.add(Dense(X.shape[1], input_dim=nhidden))

mod.compile(loss='mean_squared_error', optimizer='sgd')
mod.fit(Xw, Xw, epochs=10, verbose=0)

In [ ]:
from keras import backend as K

l1 = mod.layers[0]

coef = K.get_value(l1.kernel)

plt.pcolormesh(coef)

In [ ]:
mod.evaluate(Xw, Xw)

this doesn't seem to work very well. In theory this sample should work very well, since it is a convex problem. Maybe neural networks should be mostly used for classification type problems.

In [ ]:
PCA(n_components=4)

# Principal Component Regression

In [ ]:
pipe = Pipeline([('pca', PCA(n_components=20)), ('reg', LinearRegression())])

In [ ]:
pipe.named_steps['pca'].set_params()

In [ ]:
ntrain = 600

def score(pipe, X, Y):
    
    Yt = pipe.predict(X)
    return 1 - np.mean((Yt-Y)**2)/Y.std()**2

def train(n=4, train=600):
    pipe.named_steps['pca'].set_params(n_components=n) 
    pipe.fit(Xw[:train], Xw[1:train+1])
    return score(pipe, Xw[:train], Xw[1:train+1]), score(pipe, Xw[train-1:-1], Xw[train:])

In [ ]:
n_c = np.arange(1,64)

train_error, test_error = zip(*[train(n) for n in range(1,64)])

In [ ]:
plt.plot(train_error, label='training score')
plt.plot(test_error, label='testing score')
plt.legend()
plt.ylim([0,1.0])

In [ ]:
nst = n_c[np.argmax(test_error)]

_, err  = train(n_c[np.argmax(test_error)])
print(f"Best test R2 of {err} with {nst} components")

plt.pcolormesh(pipe.named_steps['pca'].components_)